In [1]:
import typing as t
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

# Define the connection string for SQL Server (Azure)
CONN_STR = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD}"
)

In [ ]:
def get_data() -> pd.DataFrame:
    """
    Get initial data from the Azure SQL Database.

    Returns:
        pd.DataFrame: DataFrame containing the data from the database.
    """

    try:
        conn = pyodbc.connect(CONN_STR)
        print("Connected to the Azure SQL Database successfully!")

    except pyodbc.Error as e:
        print(f"Error connecting to the database: {e}")

    query = "SELECT * FROM SalesLT.customerFROM"

    try:
        df = pd.read_sql(query, conn)
        print("Data retrieved successfully!")
        conn.close()
    except Exception as e:
        print(f"Error reading data: {e}")
        return pd.DataFrame()

    return df


df: pd.DataFrame = get_data()

df.head()

Connected to the Azure SQL Database successfully!


C:\Users\angel\AppData\Local\Temp\ipykernel_8984\4098523625.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data retrieved successfully!
   CustomerID  NameStyle Title FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.   Orlando         N.         Gee   None   
1           2      False   Mr.     Keith       None      Harris   None   
2           3      False   Ms.     Donna         F.    Carreras   None   
3           4      False   Ms.     Janet         M.       Gates   None   
4           5      False   Mr.      Lucy       None  Harrington   None   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8   
2    Advanced Bike Components  adventure-works\jillian0   
3       Modular Cycle Systems  adventure-works\jillian0   
4  Metropolitan Sports Supply      adventure-works\shu0   

                   EmailAddress         Phone  \
0  orlando0@adventure-works.com  245-555-0173   
1    keith0@adventure-works.com  170-555-0127   
2    donna0@adventure-wo

In [ ]:
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares the data for prediction.

    Args:
        df (pd.DataFrame): The input DataFrame containing the data.

    Returns:
        pd.DataFrame: The prepared DataFrame with the necessary features.
    """

    df = df[["FirstName", "ModifiedDate"]].sort_values(by="ModifiedDate")
    df["FirstNameLen"] = df["FirstName"].str.len()
    df["ModifiedDate"] = (
        pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    )
    df["ModifiedDate"] = df["ModifiedDate"].dt.days

    return df[["FirstNameLen", "ModifiedDate"]].reset_index(drop=True)


In [ ]:
df2 = prepare_data(df)
df2.head(100)

    FirstNameLen  ModifiedDate
0              5             0
1              7             0
2              5             0
3              7             0
4              7             0
..           ...           ...
95             5            31
96             6            31
97             5            31
98             5            31
99             5            31

[100 rows x 2 columns]


In [ ]:
# Assign dates to the ModifiedDate column
df3 = df2.copy()

df3["ModifiedDate"] = pd.to_datetime(
    df2["ModifiedDate"],
    unit="D",
    origin="2005-07-01",
)

df3.head(100)

,FirstNameLen,ModifiedDate
0,5,2005-07-01
1,7,2005-07-01
2,5,2005-07-01
3,7,2005-07-01
4,7,2005-07-01
...,...,...
95,5,2005-08-01
96,6,2005-08-01
97,5,2005-08-01
98,5,2005-08-01


In [16]:
df3.tail()

,FirstNameLen,ModifiedDate
842,7,2009-05-16
843,8,2009-05-16
844,8,2009-05-16
845,3,2009-05-16
846,3,2009-05-16


In [3]:
from datetime import date, timedelta


def int_to_date(day_int: int, start_date: t.Optional[date] = None) -> str:
    """
    Convert an integer to a date string in the format YYYY-MM-DD.

    This function is specifically designed to this dataset
    """

    if start_date is None:
        start_date = date(2005, 7, 1)

    result = start_date + timedelta(days=day_int)
    result = result.strftime("%Y-%m-%d")
    return result

In [4]:
list_example = [1, 2, 3, 4, 5]

list_dates = list()
for i, value in enumerate(list_example):
    list_dates.append(int_to_date(value))

list_dates

['2005-07-02', '2005-07-03', '2005-07-04', '2005-07-05', '2005-07-06']